In [1]:
from nanotag.data import NanotagData, ImageFileCollection, Summary
from nanotag.image import ImageSeries, GaussianFilterSlider
from nanotag.tags import PointTags, PointTagSeries
from nanotag.timeline import Timeline, TimelineTags, EditTimelineTags
from nanotag.canvas import Canvas, ToolBox
from nanotag.tools import EditPointTags, ResetView, BoxZoomTool, PanZoomTool
from nanotag.utils import link
from nanotag.histogram import Histogram

from bqplot import ColorScale, Scatter, Figure
import ipywidgets as widgets
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
root_directory = '/data/'
analysis_file = 'ml_analysis.json'
filt = '/data/*.tif'

In [3]:
image_file_collection = ImageFileCollection(filter=filt)

images = ImageSeries()

gaussian_filter = GaussianFilterSlider(max=10)
images.filters = [gaussian_filter]

point_tags = PointTagSeries(data_fields=('sublattice', 'intensities', 'normalized_intensities'))
defect_tags = PointTagSeries(data_fields=('type',))

point_tags.point_tags.color_scheme = 'plasma'
defect_tags.point_tags.color_scheme = 'category'

area_tags = TimelineTags(row='area', data_fields=('area',), enable_move=False, color_scale=ColorScale(min=0, max=1))
#start_tags = TimelineTags(row='start', default_t=np.zeros((1,), dtype=np.int64), color_scale=ColorScale(min=0, max=1))

nanotag_data = NanotagData(root_directory=root_directory,
                           read_file = analysis_file,
                           write_file = analysis_file,
                           tags = {'points' : point_tags, 'defects': defect_tags, 'area':area_tags}
                           )

canvas = Canvas()
canvas.tags = [point_tags, defect_tags]
canvas.image = images

timeline = Timeline()
timeline.tags = [area_tags]

toolbox = ToolBox(canvas)
toolbox.button_width = '80px'
toolbox.tools = {'Reset' : ResetView(),
                'BoxZoom' : BoxZoomTool(),
                'Pan' : PanZoomTool(),
                'S defect' : EditPointTags(defect_tags, data_fields={'type':0}),
                'Mo defect' : EditPointTags(defect_tags, data_fields={'type':1}),
               }

link((nanotag_data, 'root_directory'), (image_file_collection, 'root_directory'))

link((image_file_collection, 'images'), (images, 'images'), check_broken=False)
link((image_file_collection, 'relative_path'), (nanotag_data, 'identifier'), check_broken=False)

link((images, 'length'), (timeline, 'num_frames'))

link((timeline, 'frame_index'), (images, 'index'))
link((timeline, 'frame_index'), (point_tags, 'index'))
link((timeline, 'frame_index'), (defect_tags, 'index'));

In [4]:
def area_changed(change):
    if len(area_tags.area) > 0:
        timeline.frame_index = int(np.argmax(area_tags.area > .5))

area_tags.observe(area_changed, 'area')

In [5]:
#defect_tags.point_tags.color_scale.colors = ['cyan', 'lime']
defect_tags.point_tags.color_scale.min= 0
defect_tags.point_tags.color_scale.max= 10

In [6]:
hist1 = Histogram(height=220, min=.5, max=1.5, label='S sublattice', adjust_x_scale=False)
hist1.bins=30
hist2 = Histogram(height=220, min=.5, max=1.5, label='Mo sublattice', adjust_x_scale=False)
hist2.bins=30


def update_func():
    summary = {}
    
    summary['sampling'] = None
    summary['first_defect_frame'] = None
    summary['first_defect_type'] = None
    summary['first_defect_x'] = None
    summary['first_defect_y'] = None
    summary['path'] = image_file_collection.relative_path
    
    if len(area_tags.area) > 0:
        summary['start_frame'] = int(np.argmax(area_tags.area > .5))
    else:
        summary['start_frame'] = 0
    
    try:
        summary['sampling'] = nanotag_data.selected_data['sampling'][str(images.index)]
    except KeyError:
        pass
    
    if len(defect_tags.series.keys()) == 0:
        return summary
        
    has_defect = [i for i, value in defect_tags.series.items() if len(value['y']) > 0]
    
    if len(has_defect) == 0:
        return summary

    first_frame = min(has_defect)
    first_defect = np.argmin(defect_tags.series[first_frame]['y'])
    first_type = defect_tags.series[first_frame]['type'][first_defect]

    summary['first_defect_frame'] = int(first_frame)
    summary['first_defect_x'] = defect_tags.series[first_frame]['x'][first_defect]
    summary['first_defect_y'] = defect_tags.series[first_frame]['y'][first_defect]
    summary['first_defect_type'] = ['S', 'Mo'][int(first_type)]
    
    return summary

summary = Summary(update_func, write_file='summary.json')

link((nanotag_data, 'identifier'), (summary, 'append_key'))
link((nanotag_data, 'root_directory'), (summary, 'path'))

defect_tags.observe(lambda *args: summary._update(), 'series')

def threshold_changed(*args):
    series = {}

    for key, value in point_tags.series.items():
        
        is_under = np.zeros(len(value['normalized_intensities']), dtype=bool)
        
        is_under[np.array(value['sublattice']) == 0] = np.array(value['normalized_intensities'])[np.array(value['sublattice']) == 0] < hist1.lower
        is_under[np.array(value['sublattice']) == 1] = np.array(value['normalized_intensities'])[np.array(value['sublattice']) == 1] < hist2.lower
        
        series[key] = {'x':[],'y':[],'type':[]}
        
        if sum(is_under) == 0:
            continue
        
        series[key]['x'] = np.array(value['x'])[is_under].tolist()
        series[key]['y'] = np.array(value['y'])[is_under].tolist()
        series[key]['type'] = np.array(value['sublattice'])[is_under].tolist()
    
    defect_tags.series = series
    defect_tags.update_current(defect_tags.index)
    summary._update()

def update_histrograms(change):
    hist1.sample = point_tags.point_tags.normalized_intensities[point_tags.point_tags.sublattice == 0]
    hist2.sample = point_tags.point_tags.normalized_intensities[point_tags.point_tags.sublattice == 1]
    threshold_changed()

point_tags.point_tags.observe(update_histrograms, 'normalized_intensities')
point_tags.point_tags.observe(threshold_changed, 'normalized_intensities')

hist1.observe(threshold_changed, 'lower')
hist2.observe(threshold_changed, 'lower')

histograms = widgets.VBox([hist1, hist2])

In [7]:
tools = widgets.VBox([nanotag_data, image_file_collection, images, gaussian_filter, point_tags, defect_tags, toolbox])

tab = widgets.Tab([tools, histograms])
tab.set_title(0, 'Tools')
tab.set_title(1, 'Histograms')

app = widgets.HBox([widgets.VBox([canvas, timeline]),
                    tab
              ,
             widgets.VBox([summary])
             
             ])

image_file_collection._load_paths()
nanotag_data.read_data()

area_changed(None)
summary._update()

app

ValueError: path is on mount 'D:', start on mount 'C:'

In [7]:
from nanotag.events import KeyEvents

events = KeyEvents(app)


def toggle_point_visible(*args):
    point_tags.point_tags.visible = not point_tags.point_tags.visible

def toggle_defects_visible(*args):
    defect_tags.point_tags.visible = not defect_tags.point_tags.visible

    
events.callbacks = {'ArrowLeft' : timeline.previous_frame,
                    'ArrowRight' : timeline.next_frame,
                    'ArrowUp' : image_file_collection.next_file,
                    'ArrowDown' : image_file_collection.previous_file,
                    'r' : canvas.reset,
                    'z' : lambda : toolbox.toggle_tool('BoxZoom'),
                    'p' : lambda : toolbox.toggle_tool('Pan'),
                    'l' : toggle_point_visible,
                    'd' : toggle_defects_visible,
                   }